# LGM DATA SCIENCE INTERN

# Name : Akshat Singh


## NEXT WORD PREDICTION

In [2]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)
import warnings as wg
wg.filterwarnings("ignore")
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dense,Activation,Dropout,RepeatVector
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

## Loading The data

In [3]:
text = open("C:\\Users\\91829\\Desktop\\1661-0.txt", encoding ="utf8").read().lower()
print('corpus length:', len(text))

corpus length: 581888


# Data Preprocessing

In [4]:
character  =  sorted(list(set(text)))
char_indices = dict((c,i) for i , c in enumerate (character))
indices_char = dict((i,c) for i , c in enumerate (character))
print(f'unique chars : {len(character)}')

unique chars : 73


In [5]:
# chunk 40 characters with 3 sequences
seq_len = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - seq_len,step):
    sentences.append(text[i: i+ seq_len])
    next_chars.append(text[i+ seq_len])
print(f'num training examples: {len(sentences)}')

num training examples: 193950


In [6]:
X = np.zeros((len(sentences) ,seq_len,  len(character)), dtype = np.bool)
Y = np.zeros((len(sentences) , len(character)), dtype = np.bool)
for i , sentence in enumerate (sentences):
    for t ,char in enumerate (sentence):
        X[i, t, char_indices[char]] = 1
    Y[i , char_indices[next_chars[i]]]
    
sentences[124]
next_chars[100]


'e'

In [7]:
X[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True])

In [8]:
Y[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [9]:
X.shape

(193950, 40, 73)

In [10]:
Y.shape

(193950, 73)

# Creatiing the model

In [11]:
model  = Sequential()
model.add(LSTM(500, input_shape = (seq_len, len(character))))
model.add(Dense(len(character)))
model.add(Activation('softmax'))

## Model Summary

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 500)               1148000   
                                                                 
 dense (Dense)               (None, 73)                36573     
                                                                 
 activation (Activation)     (None, 73)                0         
                                                                 
Total params: 1,184,573
Trainable params: 1,184,573
Non-trainable params: 0
_________________________________________________________________


## Training our model

In [12]:
optimizer  = RMSprop(learning_rate = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
history = model.fit(X,Y,
                   validation_split = 0.01,
                   batch_size = 128,
                   epochs = 5,
                   shuffle  = True).history

Epoch 1/5
1501/1501 [==============================] - 1578s 1s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
1501/1501 [==============================] - 1319s 878ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/5
1501/1501 [==============================] - 1379s 919ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/5
1501/1501 [==============================] - 1315s 876ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/5
1501/1501 [==============================] - 1334s 889ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


## Saving our model

In [14]:
model.save('nextword.h5')
pickle.dump(history, open("history.p", "wb"))

## loading our model

In [15]:
model = load_model('nextword.h5')
pickle.load(open("history.p", "rb"))

{'loss': [0.0, 0.0, 0.0, 0.0, 0.0],
 'accuracy': [0.0, 0.0, 0.0, 0.0, 0.0],
 'val_loss': [0.0, 0.0, 0.0, 0.0, 0.0],
 'val_accuracy': [0.0, 0.0, 0.0, 0.0, 0.0]}

## Model Evaluation

In [16]:
loss_and_acc = model.evaluate(X,Y)
print("Test Loss", loss_and_acc[0])
print("Test Accuracy", loss_and_acc[1])

6061/6061 [==============================] - 1212s 200ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Test Loss 0.0
Test Accuracy 0.0


### Plotting training and validation plots

In [13]:
acc  = history['accuracy']
val_acc = history['val_accuracy']
loss = history['loss']
val_loss = history['val_loss']
epochs = range(len(acc))

fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,6))
ax[0].plot(epochs,acc,'Y', label  ='Training Accuracy')
ax[0].plot(epochs, val_acc, 'g', label = 'Validation accuracy')
ax[0].legend(loc =0)

ax[1].plot(epochs,loss,'Y', label  ='Training loss')
ax[1].plot(epochs, val_loss, 'g', label = 'Validation loss')
ax[1].legend(loc= 0)

plt.suptitle('Training and validation')
plt.show()


NameError: name 'history' is not defined

In [14]:
def  prepare_input(text):
    x =np.zeros((1,seq_len, len(character)))
    for t ,char in enumerate(text):
        x[0, t , char_indices[char]] = 1
        
    return x
prepare_input("This is an example of input of our model".lower())


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

## Functions to get next probable characters

In [15]:
def sample(preds,top_n = 3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [16]:
def predict_completion(text):
    original_text = text
    generated =  text
    completion = ''
    while True:
        x  = prepare_input(text)
        preds= model.predict(x, verbose = 0)[0]
        next_index = sample(preds, top_n = 1)[0]
        next_char  = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        
        if len(original_text + completion) +2 > len(original_text) and next_char ==' ':
            return completion

In [27]:
def predict_completions(text, n= 3):
    X = prepare_input(text)
    preds= model.predict(X,verbose = 0)[0]
    next_indices = sample(preds,n)
    return [indices_char[idx] + predict_completion(text[1:] + indices_char[idx]) for idx in next_indices]

# Prediction

In [28]:
quotes = [
    "Life is what happens when you're busy making other plans." ,
    "Many of life’s failures are people who did not realize how close they were to success when they gave up."
    "Your time is limited, so don’t waste it living someone else’s life. Don’t be trapped by dogma – which is living with the results of other people’s thinking."
    "Money and success don’t change people; they merely amplify what is already there."
]

In [ ]:
for q in quotes:
    seq = q[:18].lower()
    print(seq)
    print(predict_completions(seq, 5))
    print()

### Model has been successfully tested